In [ ]:
from bs4 import BeautifulSoup
import urllib2
#import selenium

In [ ]:
#soup = BeautifulSoup(open("/Network/Servers/pg1.phys.columbia.edu/Users/Shared/vg2337/Downloads/ibm-20140630.xml"),"lxml-xml")
url = "https://www.sec.gov/cgi-bin/viewer?action=view&cik=51143&accession_number=0001047469-15-001106"
page = urllib2.urlopen(url)
soup = BeautifulSoup(page,"lxml")
import re

In [ ]:
for i in soup.find_all(class_='xbrlviewer'):
    #print i.text,i.attrs['href']
    match = re.search("loadReport\(([0-9]+)\)",i.attrs['href'])
    if match is not None:
        print i.text,match.group(1)

In [ ]:
page2 = urllib2.urlopen("https://www.sec.gov/Archives/edgar/data/51143/000104746915001106/R2.htm")
soup2 = BeautifulSoup(page2,"lxml")

In [ ]:
data = {}
curr_abstract = ''
subheading = []
import re
for f in soup2.find_all(class_="pl"):
    fmatch = re.search("defref\_((us-gaap|ibm)\_[a-zA-Z]+)",f.a['onclick']).group(1).split("_")
    
    match  = re.search("Abstract",fmatch[1])
    if match is not None:
        curr_abstract = f.text
   
    if data.has_key(curr_abstract):
        data[curr_abstract].append({ f.text : fmatch })
    else:
        data[curr_abstract] = [{ f.text : fmatch },]
    


In [ ]:
%pprint

In [ ]:
import json

In [ ]:
json.dumps(data)

In [ ]:
from selenium import webdriver
driver = webdriver.PhantomJS()

In [ ]:
driver.get("https://www.sec.gov/cgi-bin/viewer?action=view&cik=51143&accession_number=0001047469-15-001106")

In [ ]:
soup = BeautifulSoup(driver.page_source,"lxml")
print soup

In [ ]:
element    = driver.find_element_by_id("menu")
accordians = element.find_elements_by_class_name("accordion")
#accordians.click()
print element.text.split("\n")
len(accordians)
fstatements = filter(lambda x : x.text == "Financial Statements",accordians)[0]
driver.get_screenshot_as_file("/Users/vgenty/Desktop/aho.png")

In [ ]:
print fstatements.click()
driver.get_screenshot_as_file("/Users/vgenty/Desktop/aho.png")

In [ ]:
element    = driver.find_element_by_id("menu_cat2")
accordians = element.find_elements_by_class_name("xbrlviewer")
print element.text.split("\n")
len(accordians)

In [ ]:
element    = driver.find_element_by_id("menu")
reports    = element.find_elements_by_class_name("xbrlviewer")

In [ ]:
for a in reports: 
    print a.get_attribute("text"), a.get_attribute("href")

In [ ]:
driver.execute_script("loadReport(2);")

In [ ]:
driver.save_screenshot("/Users/vgenty/Desktop/current.png")

In [ ]:
soup = BeautifulSoup(driver.page_source,"lxml")

In [ ]:
report = soup.find(id="reportDiv")

In [ ]:
accordians

In [ ]:
contexts = {}
for context in xbrl_contexts: 
    period = context.findChildren(name='period')[0]
    
    if period.startDate is not None:
        contexts[context.attrs['id']] = {'period' : {'startDate' : period.startDate.contents[0],
                                                     'endDate' : period.endDate.contents[0]
                                                },
                                          'type'    : 'range'}
    if period.instant is not None:
        contexts[context.attrs['id']] = {'period' : {'Date' : period.instant.contents[0] },
                                          'type'   : 'instant'}

In [ ]:
import pandas as pd
contexts_df=pd.DataFrame(contexts)

In [ ]:
contexts_df

In [ ]:
#aho = soup.findAll(id=re.compile('ID_[0-9]+'))[0]
#aho1 = soup.findAll(lambda tag : tag.prefix == 'dei')
aho2 = soup.findAll(lambda tag : tag.prefix == 'us-gaap')

In [ ]:
document_info = {}
for dei in soup.findAll(lambda tag : tag.prefix == 'dei' and re.search('ID_[0-9]+',tag.attrs['id']) is not None):
    document_info[dei.name] = {'attrs' : dei.attrs , 'value' : dei.contents}

In [ ]:
document_df = pd.DataFrame(document_info)

In [ ]:
document_df

In [ ]:
gaap_info = {}
for gaap in soup.findAll(lambda tag : tag.prefix == 'us-gaap' and re.search('ID_[0-9]+',tag.attrs['id']) is not None and re.search('TextBlock$',tag.name) is not None):
    gaap_info[gaap.name] = {'attrs' : gaap.attrs , 'value' : gaap.contents}

In [ ]:
gaap_df = pd.DataFrame(gaap_info)

In [ ]:
gaap_df

In [ ]:
special_info = {}
for special in soup.findAll(lambda tag : tag.prefix == 'ibm' and re.search('ID_[0-9]+',tag.attrs['id']) is not None):
    special_info[special.name] = {'attrs' : special.attrs , 'value' : special.contents}

In [ ]:
special_df = pd.DataFrame(special_info)

In [ ]:
special_df